# Visualisierung

Wenn du in Jupyter-Notebooks viel mit Daten arbeitest, spielt die *Darstellung der Daten* automatisch eine große Rolle. Wenn du intuitive Visualisierungen findest, wird es dir leicht fallen, den Sinn der Daten schnell zu vermitteln.

In diesem Notebook wirst du die gleichen Daten verwenden wie im letzten, nämlich die Konsumindikatoren von *Eurostat*. Im ersten Schritt lädst du die Daten herunter und benennst die Spalten um:

In [ ]:
!pip install eurostat

In [ ]:
import pandas as pd
import eurostat
df = eurostat.get_data_df("ei_bsco_m")
df = df.rename(columns={"geo\\TIME_PERIOD": "country"})

Anschließend änderst du den Typ der Monatsspalten auf ein Datum:

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f.split("-")[0] + "-" + f.split("-")[1] + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

*Eurostat* verwendet viele Abkürzungen, bietet dazu aber auch Erklärungen an. Manchmal ist es besser, wenn du die Länder mit ihren echten Namen darstellst. `Germany` ersetzt du hier von Hand, weil sonst ein sehr langer Name (`Germany (until 1990 former territory of the FRG)`) verwendet wird.

In [ ]:
rg = eurostat.get_dic("ei_bsco_m", 'geo')
realnames_g = { k: v for (k, v) in rg }
realnames_g['DE'] = "Germany"
realnames_g['EA19'] = "EURO"
realnames_g['EU27_2020'] = "EU"

## Zeitserien

Nun hast du alles bereit, um die erste *Zeitserie* zu visualisieren. Zeitserien werden (fast) immer als *Liniendiagramme* dargestellt. Dazu muss du `pandas` gar nicht sagen, das passiert ganz automatisch: 

In [ ]:
de_cci = df[(df["country"] == "DE") & 
            (df["indic"] == "BS-CSMCI") & 
            (df["s_adj"] == "SA")]
de_cci = de_cci[[c for c in de_cci.columns 
                   if isinstance(c, datetime) and (c.year>=2019)]]
de_cci.T.plot()

## Multi-Zeitserien

Der Konsumindikator für Deutschland ist zwar interessant, aber spannender ist es, wenn du unterschiedliche Länder vergleichen kannst. Mithilfe von `isin` kannst du unterschiedliche Optionen zulassen. Den Zeitraum wählst du jetzt ab 2000, um noch mehr Konjunkturkrisen und ihre Auswirkungen sehen zu können:

In [ ]:
# Länder wählen
cci_mix = df[(df["country"].isin(["DE", "EA19", "SE"])) & 
             (df["indic"] == "BS-CSMCI") & 
             (df["s_adj"] == "SA")]

# nur bestimmte Spalte behalten und transponieren (Zeit nach unten)
cci_mix = cci_mix[[c for c in cci_mix.columns 
                     if (isinstance(c, datetime) and c.year>=2000) or c == "country"]].set_index("country").transpose()

In [ ]:
cci_mix.plot()

## Barplot

Liniendiagramme eignen sich sehr gut, wenn die x-Achse ein Zeit darstellt. Wenn du kategorische Informationen auf der x-Achse verwenden willst, sind *Barplots* besser geeignet.

Als Beispiel stellst du hier die Verteilung des Konsumindikators für den 1.1.2021 über die unterschiedlichen Länder dar:

In [ ]:
cci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
cci = cci[["country", datetime(2022,11,1)]].set_index("country")
# echte Namen verwenden
cci.index = [realnames_g[i] for i in cci.index]
cci.plot.barh(figsize=(9, 9))

Wenn du möchtest, kannst du die Werte auch noch absteigend sortieren:

In [ ]:
cci.sort_values(datetime(2022,11,1)).plot.barh(figsize=(9, 9))

## Histogramm

Wenn du dich für die Verteilung der Werte interessiert, ist das *Histogramm* ein geeignetes Instrument zur Visualisierung. Es sieht einem *Barplot* sehr ähnlich, allerdings *zählt* es immer Werte:

In [ ]:
cci.plot.hist(bins=20)

Die Werte zwischen -20 und -20 sind also am häufigsten.

Die Schritte auf der x-Achse kannst du mit dem Parameter `bins` beeinflussen. Probier das einfach mal aus!

## Boxplots

Oft wirst du dich auch für die *Streuung der Werte* interessieren. Dafür eignen sich sog. *Boxplots*. Als Ausgangsbasis betrachtest du einen einzelnen Konsumindikator für viele Länder:

In [ ]:
csmci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
csmci

Die Daten haben leider nicht das richtig Format für den Boxplot. Dazu müssen die Datensätze *untereinander* stehen. 
Du brauchst nun das Gegenteil von `pivot`, das sich in `pandas` mit der Funktion `melt` erreichen lässt. Du möchtest als Spalten `country`  (schon da) und `month` verwenden (das waren bisher individuelle Spalten). Dich interessiert nur der Wert `csmci`:

In [ ]:
columns = [c for c in csmci.columns if isinstance(c, datetime) and (c.year>=2010)]

csmci_unwrapped = pd.melt(csmci[["country"] + columns].dropna(), 
                          id_vars=["country"], var_name="month", value_name="csmci")
csmci_unwrapped

Diese Daten kannst du nun mithilfe von `seaborn` als Boxplot darstellen. Du erkennst den Median, Quantile, Extremwerte und Ausreißer.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
csmci_unwrapped["country"] = [realnames_g[i] for i in csmci_unwrapped["country"]]
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, palette="viridis")

Eine etwas schönere Darstellung erreichst du durch die Sortierung der Länder nach den Medianen des Konsumindikators:

In [ ]:
labels = csmci_unwrapped.groupby("country").agg({"csmci": "median"}).sort_values("csmci").index.values
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

## Korrelationen

Wie du in den Tabellen gesehen hast, gibt es sehr viele unterschiedliche Konsumindikatoren. In der Datenanalyse wirst du oft mit der Fragestellung konfrontiert, ob diese alle unabhängig voneinander sind. Das kann eine *Korrelationsanalyse* klären.

Dazu betrachtest du die Konsumindikatoren für Deutschland und transponierst das Ergebnis (die Zeiten sollen in den Zeilen stehen):

In [ ]:
de = df[(df["s_adj"] == "SA") & (df["country"] == "DE")].set_index("indic")[columns].transpose()
de

Ein sog. *Scatterplot* gibt dir ein erstes Verständnis, ob und wie die Werte korreliert sein könnten:

In [ ]:
de.plot.scatter(x="BS-SFSH", y="BS-CSMCI")

Etwas besser kannst du das noch mit `seaborn` darstellen. Hier erhältst du noch eine Regressionsgerade und die Verteilung der Indikatoren als Histogramm:

In [ ]:
sns.jointplot(x=de["BS-SFSH"], y=de["BS-CSMCI"], scatter=False, kind="reg")

Wenn du die Abhängigkeit noch genauer berechnen willst, kannst du die lineare Regression durch `scipy` durchführen lassen:

In [ ]:
from scipy.stats import linregress
linregress(de["BS-SFSH"], de["BS-CSMCI"])

## Geodaten

Bisher hast du eine wesentliche Dimension der Daten noch gar nicht berücksichtigt, nämlich die Geografie. Zum Glück geht das in Python auch ziemlich einfach, es gibt eine gute Integration in `pandas`, die sich `geopandas` nennt:

In [ ]:
!pip install geopandas

Konturen für Länder findest du im sog. [GeoJSON](https://de.wikipedia.org/wiki/GeoJSON)-Format. Wir haben die GeoJSON-Daten für Europa schon mal zusammengestellt:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f europe.geo.json || wget https://github.com/heiseacademy/ml-python/raw/main/01-jupyter/europe.geo.json")

In [ ]:
import geopandas
bl_geo = geopandas.read_file("europe.geo.json")
bl_geo.plot(figsize=(10,10))

Das sieht schon ganz gut aus! Zum Glück ist der `GeoDataFrame` auch eine Art `DataFrame`, so dass du ihn problemlos mittels `merge` mit dem `DataFrame` der Konsumindikatoren verbinden kannst. Da die Spalten nicht die gleichen Namen haben, musst du einmal `country` und einmal `iso_a2` angeben. Der `outer`-Join führt dazu, dass keine Länder wegfallen, die nicht zur EU gehören.


In [ ]:
hm = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
ghm = pd.merge(hm, bl_geo, left_on="country", right_on='iso_a2', how="outer")
ghm

Du siehst, dass sich eine neue Spalte `geometry` ergeben hat. Um diese wieder als Geodaten plotten zu können, musst du sie nun wieder in einen `GeoDataFrame` wandeln:

In [ ]:
geopandas.GeoDataFrame(ghm).plot(column=datetime(2021, 1, 1), 
         legend=True, 
         legend_kwds={'orientation': "horizontal"}, 
         missing_kwds={
           "color": "lightgrey",
    },
         figsize=(10,10))

Das Ergebnis sieht richtig toll aus! Im Vergleich zu den bisherigen Daten kannst du auch gut das Nord-Süd-Gefälle in der EU erkennen, nur Serbien tanzt etwas aus der Reihe.